<a href="https://colab.research.google.com/github/AvanishSiva/hands-on-machine-learning/blob/main/Chapter_6%20/DecisionTreeExercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise

#7. Train and Tune Moons dataset using decision tree


In [1]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

In [2]:
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [5]:
dtc = DecisionTreeClassifier(max_depth=2, random_state=42)

In [8]:
param_grid = {
    'max_depth': list(range(2, 10)),
    'max_leaf_nodes' : list(range(2, 200))
}
gscv = GridSearchCV(dtc, param_grid=param_grid, cv=5)

In [9]:
gscv.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=2, random_state=42),
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...]})

In [10]:
gscv.best_estimator_

DecisionTreeClassifier(max_depth=8, max_leaf_nodes=31, random_state=42)

In [11]:
gscv.best_score_

0.8600000000000001

In [12]:
y_pred  = gscv.predict(X_test)

In [13]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.8644

##8. Grow a Forest

generating 1000 subsets of the traing set each contianing 100 instances

In [14]:
from sklearn.model_selection import ShuffleSplit


In [16]:
Sfl = ShuffleSplit(n_splits=1000, train_size=len(X_train) - 100, random_state=42)

In [17]:
mini_sets = []

In [18]:
for mini_train_set, mini_test_set in Sfl.split(X_train):
    X_mini_train = X_train[mini_train_set]
    y_mini_train = y_train[mini_train_set]
    mini_sets.append((X_mini_train, y_mini_train))

In [19]:
mini_sets

[(array([[ 1.39770031, -0.45705189],
         [-0.94732603,  0.14694238],
         [-0.40000839,  0.6591222 ],
         ...,
         [ 2.0937368 ,  0.69253162],
         [ 0.41484734, -0.65569855],
         [-0.80373287,  0.53492409]]),
  array([1, 0, 1, ..., 1, 1, 0])),
 (array([[ 0.86149414, -0.41878987],
         [-0.23165299,  0.83471509],
         [-0.20156967,  0.67045551],
         ...,
         [ 1.78573545, -0.89530865],
         [-0.57395299,  0.91968171],
         [ 0.59455219,  1.06681278]]),
  array([0, 0, 1, ..., 1, 0, 0])),
 (array([[ 0.92111162, -0.2340136 ],
         [ 0.10981661,  1.36432619],
         [ 1.36388169, -0.40272046],
         ...,
         [-0.80873905, -0.45282462],
         [ 0.74801847,  0.98610922],
         [ 1.43304384, -0.31733308]]),
  array([1, 0, 1, ..., 0, 0, 1])),
 (array([[-0.55439707,  0.70013604],
         [ 0.35428745,  0.74582457],
         [ 0.71784947,  0.68817634],
         ...,
         [-0.40861671,  0.81107099],
         [-1.241057

In [20]:
from sklearn.base import clone

forest = [clone(gscv.best_estimator_) for _ in range(1000)]

In [25]:
from sklearn.metrics import accuracy_score
accuracy =[]

for tree, (X_train_set, y_train_set) in zip(forest, mini_sets):
  tree.fit(X_train_set, y_train_set)

  y_pred = tree.predict(X_test)
  accu = accuracy_score(y_test, y_pred)
  accuracy.append(accu)

In [26]:
import numpy as np

np.mean(accuracy)

0.8611868

In [35]:
Y_pred = np.empty([1000, len(X_test)], dtype=np.uint8)
#created a empty array for [1000, length of train data]

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [36]:
Y_pred

array([[1, 1, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 1, 0, 0],
       ...,
       [1, 1, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 1, 0, 0]], dtype=uint8)

In [37]:
from scipy.stats import mode

y_predicted_maj_votes, n_votes = mode(Y_pred, axis=0)

In [38]:
y_predicted_maj_votes

array([1, 1, 0, ..., 1, 0, 0])

In [39]:
accuracy_score(y_test, y_predicted_maj_votes.reshape([-1]))

0.8644